In [ ]:
# default_exp samptools

# Sample tools

> Collection functions to support sampling.

In [ ]:
# hide
from nbdev.showdoc import *

In [ ]:
#export
import numpy as np
from scipy.special import factorial

In [ ]:
#export
def combination(n, k):
    """Vectorized function to calculate a combination"""
    return factorial(n) / (factorial(k) * factorial(n-k))

In [ ]:
#export
def binomial(k, n, p):
    """Vectorized function to calculate binomial function"""
    return combination(n,k) * p**k * (1-p)**(n-k)

In [ ]:
#export
def Wilson_std(p, N, z=1.96):
    """Wilson estimator of std of binomial distribution"""
    wilson_max = (p + z**2/(2*N) + z*np.sqrt(p*(1-p)/N+z**2/(4*N**2)))/(1+z**2/N)
    wilson_min = (p + z**2/(2*N) - z*np.sqrt(p*(1-p)/N+z**2/(4*N**2)))/(1+z**2/N)
    return wilson_max - wilson_min

def Wilson_var(p, N):
    return Wilson_std(p, N)**2

In [ ]:
#export
def Wald_var(p, N):
    """Wald variance estimator (known issues, better use Wilson)"""
    return p * (1-p) / N

def Wald_std(p, N):
    return np.sqrt(Wald_var(p, N))

In [ ]:
#export
def std_sum(Aws, pws, n_samples, var=Wilson_var):
    """Calculate standard deviation according to Gaussian error propagation 
    applied to p_L: V_L = sum_i (dp_L/dp_i)^2 * V_i"""
    return np.sqrt( np.sum( Aws[1:]**2 * var(pws[1:],n_samples), axis=0 ) )

In [ ]:
#export
def weight_cutoff(p_max, delta_max, n_circ_elems):
    """Calculate weight cutoff at p_max for delta_max."""
    delta = 1
    for w_max in range(n_circ_elems+1):
        # delta -= binom.pmf(w_max, n_circ_elems, p_max)
        delta -= binom(w_max, n_circ_elems, p_max)
        if delta < delta_max: break
    return w_max

Subset selectors

In [ ]:
#export
def balanced_SS_selector(counts, *args, **kwargs):
    """Return index of least sampled SS"""
    return np.argmin(counts)

In [ ]:
#export
def ERV_SS_selector(counts, fail_counts, var=Wilson_var, *args, **kwargs):
    """Return index of SS which yields maximum ERV"""

    p = fail_counts / counts # list of SS failure rates
    v = var(p, counts) # list of variances

    # prospective failure rates
    p_p = (fail_counts+1) / (counts+1) # next msmt yields +1 (i.e. 1)
    p_m = fail_counts / (counts+1) # next msmt yields -1 (i.e. 0)

    # prospective variances
    v_p = var(p_p, counts+1) 
    v_m = var(p_m, counts+1)

    # Calculate prospective variances
    v_prop = p*v_p + (1-p)*v_m 

    # Differences to current variance
    # The larger delta, the smaller v_prop.
    delta = v - v_prop

    # Maximize the difference
    return np.argmax(delta)

**TODO** Document & test.